In [ ]:

import pandas as pd
import math
import numpy as np
from alpha_vantage.timeseries import TimeSeries  #Need to install library -- pip install alpha_vantage
from Access_codes import AlphaVanAPIkey  # Need to get a API key
from pprint import pprint





In [ ]:

# API GET request
TCKR = str(input("What stock do you want to see?"))
ts = TimeSeries(AlphaVanAPIkey, output_format = 'pandas')
data, meta = ts.get_daily_adjusted(TCKR, outputsize ='full')

In [ ]:

columns = ['open', 'high', 'low', 'close', 'adjusted close', 'volume', 'dividend', 'split']
data.columns = columns

In [ ]:
TCKR = pd.DataFrame(data)

TCKR

TCKR.info()

In [ ]:

#Records the moving average for 20 days and 55 days

TCKR['sma20'] = TCKR['close'].rolling(window=20).mean()
TCKR['sma55'] = TCKR['close'].rolling(window=55).mean()


TCKR.info()

display(TCKR.head())

In [ ]:
risk = 0.01

entry_price = float(input('At what positon do you want to enter '))
stop_loss_long = (entry_price - (entry_price *0.02))
stop_loss_short = (entry_price + (entry_price * 0.02))
cash = float(input('How much is in account '))

risk_amount = risk * cash

position_size_short = risk_amount / stop_loss_short
position_size_long = risk_amount / stop_loss_long

print(f'position long:', {position_size_short})
print(f'position short:', {position_size_long})

In [ ]:
TCKR["signal"] = None
for i in range(len(TCKR)):
    if TCKR.iloc[i]["close"] > TCKR.iloc[i]["sma55"]:
        TCKR.at[i,"signal"] = "buy"
    elif TCKR.iloc[i]["close"] < TCKR.iloc[i]["sma20"]:
        TCKR.at[i,"signal"] = "sell"

In [ ]:
# Code to execute trades based on the signals
for i in range(len(TCKR)):
    if TCKR.iloc[i]["signal"] == "buy":
        # Code to execute a buy order
        pass
    elif TCKR.iloc[i]["signal"] == "sell":
        # Code to execute a sell order
        pass

In [ ]:
display(TCKR[['signal']].tail())

In [ ]:
TCKR.to_csv("TCKR.csv")

In [ ]:
print(stop_loss_short)
print(stop_loss_long)

In [ ]:
for stop in range(len(TCKR)):
    if TCKR.iloc[stop]["stopOrexit"] == "S2 S/L":
        # Code to execute a buy order
        pass
    elif TCKR.iloc[stop]["stopOrexit"] == "S1 S/L":
        # Code to execute a sell order
        pass    
    elif TCKR.iloc[stop]["stopOrexit"] == "S1 Exit Long":
        pass
    elif TCKR.iloc[stop]["stopOrexit"] == "S1 Exit Short":
        pass
    elif TCKR.iloc[stop]["stopOrexit"] == "S2 Exit Long":
        pass
    elif TCKR.iloc[stop]["stopOrexit"] == "S2 Exit Short":
        pass

In [ ]:
TCKR = TCKR.sort_values(by=['Date'])

display(TCKR[['signal', 'stopOrexit', 'Date', 'from_file']].tail(10))

TCKR.to_csv("TCKR.csv")


In [ ]:
def expectancy(trades):
    num_trades = len(trades)
    
    if num_trades == 0:
        return 0
    
    wins = sum(float(trade['PNL']) > 0 for trade in trades)
    PW = wins / num_trades

    AW = sum(float(trade['PNL']) for trade in trades if float(trade['PNL']) > 0) / wins if wins > 0 else 0

    AL = sum(float(trade['PNL']) for trade in trades if float(trade['PNL']) < 0) / (num_trades - wins) if num_trades - wins > 0 else 0
    
    E = (PW * AW) - ((1 - PW) * abs(AL))
    return E